# Imports

In [18]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import requests
import random
import json
import csv
import time
import folium
from folium import Map, CircleMarker, Vega, Popup, Marker
from folium.plugins import MarkerCluster, HeatMap
import LatLon23
from vincent import Bar
from math import radians, sin, cos, asin, sqrt
from ipywidgets import interact

ModuleNotFoundError: No module named 'seaborn'

# Data

## Originals

In [ ]:
data_senior = pd.read_csv('raw_data/senior_transportation.csv')
data_bus = pd.read_csv('raw_data/bus_routes.csv')
data_users = pd.read_csv('raw_data/churn_users.csv')
data_od = pd.read_csv('raw_data/churn_OD.csv')


In [2]:
data_senior

NameError: name 'data_senior' is not defined

In [3]:
data_senior.info()

NameError: name 'data_senior' is not defined

In [4]:
plot = data_senior[["County_of_Origin", "Average_Daily_SeniorPopulation_Travelling"]].groupby("County_of_Origin").mean()
plt.plot(plot)
plt.xticks(rotation='vertical')
plt.show()

NameError: name 'data_senior' is not defined

In [5]:
data_bus

NameError: name 'data_bus' is not defined

In [6]:
data_users

NameError: name 'data_users' is not defined

In [7]:
plt.plot(data_users.groupby("AgeClassDescription").mean())

NameError: name 'plt' is not defined

In [8]:
data_od

NameError: name 'data_od' is not defined

## transporlis

In [9]:
df_carr = pd.read_csv("transporlis_data/data_carristur/shapes.txt")

In [10]:
df_carr

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,21635,38.705920,-9.144050,1,NaN
1,21635,38.709667,-9.136373,2,NaN
2,21635,38.713560,-9.138840,3,NaN
3,21635,38.715790,-9.141390,4,NaN
4,21635,38.719740,-9.145020,5,NaN
5,21635,38.727190,-9.148700,6,NaN
6,21635,38.730500,-9.146710,7,NaN
7,21635,38.733920,-9.144640,8,NaN
8,21635,38.742910,-9.146840,9,NaN
9,21635,38.746110,-9.147540,10,NaN


In [11]:
df_trips = pd.read_csv("transporlis_data/data_carristur/trips.txt")

In [12]:
df_trips_carr = pd.merge(df_trips, df_carr[["shape_id", "shape_pt_lat", "shape_pt_lon"]], 
                         left_on="shape_id", right_on="shape_id")

In [13]:
df_trips_carr

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,shape_pt_lat,shape_pt_lon
0,21635,25964425,25964425,NaN,NaN,NaN,21635,38.705920,-9.144050
1,21635,25964425,25964425,NaN,NaN,NaN,21635,38.709667,-9.136373
2,21635,25964425,25964425,NaN,NaN,NaN,21635,38.713560,-9.138840
3,21635,25964425,25964425,NaN,NaN,NaN,21635,38.715790,-9.141390
4,21635,25964425,25964425,NaN,NaN,NaN,21635,38.719740,-9.145020
5,21635,25964425,25964425,NaN,NaN,NaN,21635,38.727190,-9.148700
6,21635,25964425,25964425,NaN,NaN,NaN,21635,38.730500,-9.146710
7,21635,25964425,25964425,NaN,NaN,NaN,21635,38.733920,-9.144640
8,21635,25964425,25964425,NaN,NaN,NaN,21635,38.742910,-9.146840
9,21635,25964425,25964425,NaN,NaN,NaN,21635,38.746110,-9.147540


In [14]:
df_freq = pd.read_csv("transporlis_data/data_carristur/frequencies.txt")

In [15]:
df_freq.value_counts()

trip_id   start_time  end_time  headway_secs  exact_times
25964425  08:00:00    21:00:00  1200          1              1
25964426  08:00:00    21:00:00  1200          1              1
25964427  08:00:00    19:00:00  3600          1              1
25964428  08:30:00    19:30:00  3600          1              1
dtype: int64

# Mapping

In [16]:
# map for transporlis
m = folium.Map(location=[38.71, -9.18])
    # Avoid preloading all points
# marker_cluster = MarkerCluster().add_to(m)
for c, row in df_trips_carr.iterrows():
    coor = list([row["shape_pt_lat"], row["shape_pt_lon"]])
    popup = f'Route ID: {int(row["route_id"])}'
    tooltip = f'Trip ID: {int(row["trip_id"])}'
#     print(popup)
    Marker(coor, popup=popup, tooltip=tooltip).add_to(m)

m

NameError: name 'folium' is not defined

In [17]:
# map for seniors table
m = folium.Map(location=[38.71, -9.18])
    # Avoid preloading all points
# marker_cluster = MarkerCluster().add_to(m)
for c, row in df_trips_carr.iterrows():
    coor = list([row["shape_pt_lat"], row["shape_pt_lon"]])
    popup = f'Route ID: {int(row["route_id"])}'
    tooltip = f'Trip ID: {int(row["trip_id"])}'
#     print(popup)
    Marker(coor, popup=popup, tooltip=tooltip).add_to(m)

m

NameError: name 'folium' is not defined